### MODELS

In [218]:
import os
import random
import time
from dataclasses import dataclass
import pickle

import gymnasium as gym
from gymnasium import spaces
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tyro
from stable_baselines3.common.buffers import ReplayBuffer
from torch.utils.tensorboard import SummaryWriter

from goal_task import GoalTask


@dataclass
class Args:
    exp_name: str = 'Hebbian'
    """the name of this experiment"""
    seed: int = 1
    """seed of the experiment"""
    torch_deterministic: bool = True
    """if toggled, `torch.backends.cudnn.deterministic=False`"""
    cuda: bool = True
    """if toggled, cuda will be enabled by default"""
    track: bool = False
    """if toggled, this experiment will be tracked with Weights and Biases"""
    wandb_project_name: str = "MoA_ESN_wandb"
    """the wandb's project name"""
    wandb_entity: str = None
    """the entity (team) of wandb's project"""
    capture_video: bool = False
    """whether to capture videos of the agent performances (check out `videos` folder)"""
    save_model: bool = False
    """whether to save model into the `runs/{run_name}` folder"""
    upload_model: bool = False
    """whether to upload the saved model to huggingface"""
    hf_entity: str = ""
    """the user or org name of the model repository from the Hugging Face Hub"""

    #Environment specific arguments
    goal_change = False
    test = False
    initial_positions = None

    # Algorithm specific arguments
    env_id: str = "GoalTask"
    """the id of the environment"""
    total_timesteps: int = 60000
    """total timesteps of the experiments"""
    learning_rate: float = 3e-4
    """the learning rate of the optimizer"""
    buffer_size: int = 64
    """the replay memory buffer size"""
    gamma: float = 0.95
    """the discount factor gamma"""
    tau: float = 0.05
    """target smoothing coefficient (default: 0.005)"""
    batch_size: int = 64
    """the batch size of sample from the reply memory"""
    policy_noise: float = 0
    """the scale of policy noise"""
    exploration_noise: float = 0
    """the scale of exploration noise"""
    learning_starts: int = 0
    """timestep to start learning"""
    policy_frequency: int = 2
    """the frequency of training policy (delayed)"""
    noise_clip: float = 0.5
    """noise clip parameter of the Target Policy Smoothing Regularization"""


def make_env(env_id, seed, idx, capture_video, run_name, initial_position=None):
    def thunk():
        if capture_video and idx == 0:
            env = gym.make(env_id, render_mode="rgb_array")
            env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        else:
            env = GoalTask(initial_position = initial_position)
        env.action_space.seed(seed)
        return env

    return thunk


# ALGO LOGIC: initialize agent here:
class QNetwork(nn.Module):
    def __init__(self, env):
        super().__init__()
        self.fc1 = nn.Linear(np.array(env.observation_space.shape).prod() + np.prod(env.action_space.shape), 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x, a):
        x = torch.cat([x, a], 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


class Actor(nn.Module):
    def __init__(self, env):
        super().__init__()
        self.fc1 = nn.Linear(np.array(env.observation_space.shape).prod(), 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc_mu = nn.Linear(32, np.prod(env.action_space.shape))
        # action rescaling
        self.register_buffer(
            "action_scale", torch.tensor((env.action_space.high - env.action_space.low) / 2.0, dtype=torch.float32)
        )
        self.register_buffer(
            "action_bias", torch.tensor((env.action_space.high + env.action_space.low) / 2.0, dtype=torch.float32)
        )

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.tanh(self.fc_mu(x))
        return x * self.action_scale + self.action_bias


class ESNLayer(nn.Module):
    def __init__(self, env, reservoir_size, spectral_radius=0.95, g=2.2, sparsity=0.1, hebb_lr=3e-4, device='cuda'):
        super(ESNLayer, self).__init__()
        
        self.device = device
        self.input_size = np.array(env.observation_space.shape).prod()
        self.reservoir_size = reservoir_size
        self.spectral_radius = spectral_radius
        self.g = g
        
        # Input weights: weights are sampled from a uniform distribution over [-0.5,0.5]
        self.W_in = torch.rand(reservoir_size, self.input_size, device=self.device) - 0.5
        
        # Reservoir weights: sparse random matrix
        self.W = torch.randn(reservoir_size, reservoir_size, device=self.device)
        self.W[torch.rand(reservoir_size, reservoir_size) > sparsity] = 0.0  # Sparsify
        self.W_mask = torch.ones((reservoir_size, reservoir_size), device=self.device)
        self.W_mask[self.W == 0] = 0

        # Scale the reservoir weights to have the desired spectral radius
        _, eigenvalues = torch.linalg.eig(self.W)
        max_eigenvalue = torch.max(torch.abs(eigenvalues))
        self.W *= self.spectral_radius / max_eigenvalue
        self.W = nn.Parameter(self.W)
        # Define the weight change for the reservoir
        self.hebb_trace = torch.zeros((256,256), device=self.device)
        self.hebb_step = 0
        self.past_activation = torch.zeros((256,256), device=self.device)
        self.delta_W = torch.zeros((256,256), device=self.device)
        self.hebb_lr =hebb_lr

        # The output weights, to be learned during training
        self.W_out = nn.Linear((reservoir_size+self.input_size), env.action_space.shape[0], device=self.device)
        
    
    def forward(self, u, hidden_state):
        # Compute the new reservoir state with leaky integration
        with torch.no_grad():
            reservoir_output = torch.matmul(self.W, hidden_state.T) + 0.2 * torch.matmul(self.hebb_trace, hidden_state.T)
            next_hidden_state = torch.matmul(self.W_in, u.T) + self.g * reservoir_output
            next_hidden_state = torch.tanh(next_hidden_state)
        self.next_hidden_state = next_hidden_state
        self.hidden_state = hidden_state
        
        # Append output prediction (linear transformation of the reservoir state)
        output = torch.tanh(self.W_out(torch.concat([next_hidden_state.T, u],dim=-1)))

        # Stack outputs over time and return
        return output
    
    def hebbian_update(self, hidden_state):
        with torch.no_grad():
            activations = (torch.matmul(self.W, hidden_state.T) + 0.2 * torch.matmul(self.hebb_trace, hidden_state.T)).T  # Shape: (batch_size, num_neurons)
            
            # Calculate weight updates for each sample in the batch
            batch_size = hidden_state.size(0)
            weight_updates = torch.zeros_like(self.hebb_trace)  # Initialize weight updates to zero

            for i in range(batch_size):
                # For each sample, calculate the Oja update rule
                y = activations[i]  # Shape: (num_neurons,)
                x_i = hidden_state[i].unsqueeze(0)  # Shape: (1, input_size)
                
                # Compute the outer product update based on Oja's rule
                # weight_updates_i = learning_rate * (y * x_i - y^2 * weights)
                weight_updates += self.hebb_lr * (
                    torch.matmul(y.unsqueeze(1), x_i) - (y ** 2).unsqueeze(1) * self.W
                )

            # Average the updates over the batch and apply to weights
            self.W.data += weight_updates / batch_size
            self.W = nn.Parameter(self.W * self.W_mask)
    
    def hebbtrace_update(self, hidden_state):
        with torch.no_grad():
            activations = (torch.matmul(self.W, hidden_state.T) + 0.2 * torch.matmul(self.hebb_trace, hidden_state.T)).T  # Shape: (batch_size, num_neurons)
            
            # Calculate weight updates for each sample in the batch
            batch_size = hidden_state.size(0)
            weight_updates = torch.zeros_like(self.hebb_trace)  # Initialize weight updates to zero

            for i in range(batch_size):
                # For each sample, calculate the Oja update rule
                y = activations[i]  # Shape: (num_neurons,)
                # Number of elements in the top 10%
                k = int(y.numel() * 0.1)

                # Get the threshold value for the top 10%
                top_10_percent_values, _ = torch.topk(y.flatten(), k)
                threshold = top_10_percent_values[-1]  # The smallest value in the top 10%

                # Create a mask where values are above or equal to the threshold
                mask = y >= threshold
                y = y * mask
                x_i = hidden_state[i].unsqueeze(0)  # Shape: (1, input_size)
                
                # Compute the outer product update based on Oja's rule
                # weight_updates_i = learning_rate * (y * x_i - y^2 * weights)
                weight_updates += self.hebb_lr * (
                    torch.matmul(y.unsqueeze(1), x_i) - (y ** 2).unsqueeze(1) * self.hebb_trace
                )

            # Average the updates over the batch and apply to weights
            self.hebb_trace.data += weight_updates / batch_size
            self.hebb_trace = self.hebb_trace * self.W_mask
            _, eigenvalues = torch.linalg.eig(self.hebb_trace)
            max_eigenvalue = torch.max(torch.abs(eigenvalues))
            self.hebb_step *= self.spectral_radius / max_eigenvalue            
        
    def get_hiddenstate(self):
        return self.hidden_state, self.next_hidden_state
    
    def reset_state(self):
        """Reset the hidden state of the ESN."""
        self.hiddenstate = torch.zeros(self.reservoir_size)
    


def inference(actor, hidden_state, args):
    trajectory = []
    for initial_postion in args.initial_positions:
        inference_env = make_env(args.env_id, args.seed, 0, args.capture_video, run_name, initial_position=initial_postion)()
        obs, _ = inference_env.reset() 
        hidden_state = hidden_state
        inference_actor = actor
        while True:
            with torch.no_grad():
                action = inference_actor(torch.Tensor(obs).to(device), torch.Tensor(hidden_state).to(device))
                action = action.cpu().numpy()
                next_obs, reward, termination, truncated, info = inference_env.step(action)
                _, next_hidden_state = inference_actor.get_hiddenstate()
                if termination:
                    break
                obs = next_obs
                hidden_state = next_hidden_state
        trajectory.append(inference_env.get_trajectory())
        inference_env.close()
    return trajectory

### INITIALIZATION

In [214]:
import stable_baselines3 as sb3

if sb3.__version__ < "2.0":
    raise ValueError(
            """Ongoing migration: run the following command to install the new dependencies:
poetry run pip install "stable_baselines3==2.0.0a1"
"""
        )

args = Args()
run_name = f"{args.env_id}__{args.exp_name}__{args.seed}__{int(time.time())}"
if args.track:
    import wandb

    wandb.init(
        project=args.wandb_project_name,
        entity=args.wandb_entity,
        sync_tensorboard=True,
        config=vars(args),
        name=run_name,
        monitor_gym=True,
        save_code=True,
    )
writer = SummaryWriter(f"runs/{run_name}")
writer.add_text(
    "hyperparameters",
    "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(args).items()])),
)

# TRY NOT TO MODIFY: seeding
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.backends.cudnn.deterministic = args.torch_deterministic

device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")

In [216]:
# env setup
torch.manual_seed(args.seed)
envs = make_env(args.env_id, args.seed, 0, args.capture_video, run_name)()
reservoir_size = 256
input_space = spaces.Box(low=-1, high=1, shape=(envs.observation_space.shape[0]+reservoir_size,), dtype=np.float32)

actor =  ESNLayer(envs, 256).to(device)
qf1 = QNetwork(envs).to(device)
qf2 = QNetwork(envs).to(device)
qf1_target = QNetwork(envs).to(device)
qf2_target = QNetwork(envs).to(device)
target_actor = ESNLayer(envs, 256).to(device)
target_actor.load_state_dict(actor.state_dict())
qf1_target.load_state_dict(qf1.state_dict())
qf2_target.load_state_dict(qf2.state_dict())
q_optimizer = optim.Adam(list(qf1.parameters()) + list(qf2.parameters()), lr=args.learning_rate)
actor_optimizer = optim.Adam(list(actor.parameters()), lr=args.learning_rate)

envs.observation_space.dtype = np.float32



rb = ReplayBuffer(
    args.buffer_size,
    input_space,
    envs.action_space,
    device,
    handle_timeout_termination=False,
)
start_time = time.time()

### TEST RUN

In [19]:
obs, _ = envs.reset(seed=args.seed)
hidden_state = np.zeros((reservoir_size,))
input = np.concat((obs,hidden_state))
for i in range(400):
    obs = input[:envs.observation_space.shape[0]]
    hidden_state = input[envs.observation_space.shape[0]:]
    with torch.no_grad():
        actions = actor(torch.Tensor(obs).to(device), torch.Tensor(hidden_state).to(device))
        actions = actions.cpu().numpy()
    hidden_state, next_hidden_state = actor.get_hiddenstate()
    hidden_state, next_hidden_state = hidden_state.cpu(), next_hidden_state.cpu()
    next_obs, rewards, terminations, truncations, infos = envs.step(actions)
    input = input = np.concat((obs,hidden_state))
    next_input = np.concat((next_obs, next_hidden_state))
    rb.add(input, next_input, actions, rewards, terminations, infos)
    rb.sample(args.batch_size)
    with torch.no_grad():
        actor.hebbian_update(hidden_state)


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat in method wrapper_CUDA_addmv_)

### TRAIN

In [217]:
# TRY NOT TO MODIFY: start the game
obs, _ = envs.reset(seed=args.seed)
hidden_state = np.zeros((reservoir_size,))
input = np.concat((obs,hidden_state))
all_trajectory = {}
for global_step in range(args.total_timesteps):

    # SAVE MODEL: save model per 2000 global steps
    model_dir = f"runs/{run_name}/models"
    os.makedirs(model_dir, exist_ok=True)
    if global_step % 2000 == 0 or global_step == args.total_timesteps:
        model_path = f"runs/{run_name}/models/{global_step}steps.cleanrl_model"
        torch.save(actor.state_dict(), model_path)


    obs = input[:envs.observation_space.shape[0]]
    hidden_state = input[envs.observation_space.shape[0]:]

    # INFERENCE:
    if global_step % 8000 == 0 or global_step == args.total_timesteps-1:
        args.initial_positions = [(2,2), (2,10), (2,18), (10,2), (10,18), (18,2), (18,10), (18,18)]
        trajectory = inference(actor, hidden_state, args)
        all_trajectory[f'{global_step}'] = trajectory


    # ALGO LOGIC: put action logic here
    if global_step < args.learning_starts:
        actions = envs.action_space.sample()
    else:
        with torch.no_grad():
            actions = actor(torch.Tensor(obs).to(device), torch.Tensor(hidden_state).to(device))
            actions = actions.cpu().numpy().clip(envs.action_space.low, envs.action_space.high)
            hidden_state, next_hidden_state = actor.get_hiddenstate()
            hidden_state, next_hidden_state = hidden_state.cpu(), next_hidden_state.cpu()
            

    # TRY NOT TO MODIFY: execute the game and log data.
    next_obs, rewards, terminations, truncations, infos = envs.step(actions)
    input = input = np.concat((obs,hidden_state))
    next_input = np.concat((next_obs, next_hidden_state))
    # TRY NOT TO MODIFY: record rewards for plotting purposes
    if "final_info" in infos or terminations:
        print(f"global_step={global_step}, episodic_return={infos['episode']['r']}")
        writer.add_scalar("charts/episodic_return", infos["episode"]["r"], global_step)
        writer.add_scalar("charts/episodic_length", infos["episode"]["l"], global_step)     

    # TRY NOT TO MODIFY: save data to reply buffer; handle `final_observation`
    next_obs = next_obs.copy()
    if truncations:
        real_next_obs = infos["final_observation"]
    rb.add(input, next_input, actions, rewards, terminations, infos)

    # TRY NOT TO MODIFY: CRUCIAL step easy to overlook
    input = next_input

    # ALGO LOGIC: training.
    if global_step > args.learning_starts:
        data = rb.sample(args.batch_size)
        with torch.no_grad():
            next_observations = data.next_observations[:,:envs.observation_space.shape[0]]
            next_hidden_states = data.next_observations[:,envs.observation_space.shape[0]:]
            observations = data.observations[:,:envs.observation_space.shape[0]]
            hidden_states = data.observations[:,envs.observation_space.shape[0]:]
            next_state_actions = target_actor(next_observations, next_hidden_states).clamp(
                envs.action_space.low[0], envs.action_space.high[0]
            )
            qf1_next_target = qf1_target(next_observations, next_state_actions)
            qf2_next_target = qf2_target(next_observations, next_state_actions)
            min_qf_next_target = torch.min(qf1_next_target, qf2_next_target)
            next_q_value = data.rewards.flatten() + (1 - data.dones.flatten()) * args.gamma * (min_qf_next_target).view(-1)

        qf1_a_values = qf1(observations, data.actions).view(-1)
        qf2_a_values = qf2(observations, data.actions).view(-1)
        qf1_loss = F.mse_loss(qf1_a_values, next_q_value)
        qf2_loss = F.mse_loss(qf2_a_values, next_q_value)
        qf_loss = qf1_loss + qf2_loss

        # optimize the model
        q_optimizer.zero_grad()
        qf_loss.backward()
        q_optimizer.step()

        if global_step % args.policy_frequency == 0:
            actor_loss = -qf1(observations, actor(observations, hidden_states)).mean()
            actor_optimizer.zero_grad()
            actor_loss.backward()
            actor_optimizer.step()
            actor.hebbtrace_update(hidden_states)
            #actor.hebbian_update(hidden_states)
            

            # update the target network
            for param, target_param in zip(actor.parameters(), target_actor.parameters()):
                target_param.data.copy_(args.tau * param.data + (1 - args.tau) * target_param.data)
            for param, target_param in zip(qf1.parameters(), qf1_target.parameters()):
                target_param.data.copy_(args.tau * param.data + (1 - args.tau) * target_param.data)
            for param, target_param in zip(qf2.parameters(), qf2_target.parameters()):
                target_param.data.copy_(args.tau * param.data + (1 - args.tau) * target_param.data)

        if global_step % 100 == 0:
            writer.add_scalar("losses/qf1_values", qf1_a_values.mean().item(), global_step)
            writer.add_scalar("losses/qf2_values", qf2_a_values.mean().item(), global_step)
            writer.add_scalar("losses/qf1_loss", qf1_loss.item(), global_step)
            writer.add_scalar("losses/qf2_loss", qf2_loss.item(), global_step)
            writer.add_scalar("losses/qf_loss", qf_loss.item() / 2.0, global_step)
            writer.add_scalar("losses/actor_loss", actor_loss.item(), global_step)
envs.close()
writer.close()

with open(f"runs/{run_name}/all_trajectory.pkl", "wb") as file:
    pickle.dump(all_trajectory, file)

global_step=199, episodic_return=-0.38000000000000017
global_step=399, episodic_return=-0.26000000000000006
global_step=599, episodic_return=-0.4000000000000002
global_step=799, episodic_return=-0.17
global_step=999, episodic_return=-0.11999999999999998
global_step=1199, episodic_return=-0.5100000000000002
global_step=1387, episodic_return=0.6399999999999999
global_step=1529, episodic_return=0.99
global_step=1729, episodic_return=-0.8600000000000005
global_step=1929, episodic_return=-0.22000000000000006
global_step=2129, episodic_return=-0.07
global_step=2329, episodic_return=-0.22000000000000006
global_step=2441, episodic_return=0.97
global_step=2641, episodic_return=0
global_step=2841, episodic_return=-0.15
global_step=3041, episodic_return=-0.10999999999999999
global_step=3241, episodic_return=-0.4300000000000002
global_step=3377, episodic_return=0.92
global_step=3577, episodic_return=-0.3200000000000001
global_step=3777, episodic_return=0
global_step=3880, episodic_return=1
global_

KeyboardInterrupt: 

In [212]:
actor.hebb_trace

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000, -0.0000, -0.0000,  ..., -0.1560, -0.0000,  0.1560],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0')

### PLOT

In [ ]:
import matplotlib.pyplot as plt
fig_dir = f'runs/{run_name}/figures'
os.makedirs(fig_dir, exist_ok=True)
for trajectories in all_trajectory:
    step = trajectories
    trajectories = all_trajectory[f'{trajectories}']
    fig, ax = plt.subplots(figsize=(10, 10))
    colors = plt.cm.tab10(np.linspace(0, 1, len(trajectories)))
    for i, (trajectory, color) in enumerate(zip(trajectories, colors)):
        ax.plot(trajectory[:, 0], trajectory[:, 1], color=color)
        ax.scatter(trajectory[:, 0], trajectory[:, 1], color=color, s=10)
    circle = plt.Circle((10, 10), radius=2, color='black', fill=False, linestyle='-', linewidth=2)
    ax.add_patch(circle)
    plt.xlim(0, 20)
    plt.ylim(0, 20)
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.title('Agent Trajectory in 2D Environment')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{fig_dir}/{step}.png')

### RANDOM SEEDS

In [219]:
import stable_baselines3 as sb3
for i in range(10):
    if sb3.__version__ < "2.0":
        raise ValueError(
                """Ongoing migration: run the following command to install the new dependencies:
    poetry run pip install "stable_baselines3==2.0.0a1"
    """
            )

    args = Args()
    args.exp_name = 'HebbianSpectral'
    args.seed = random.randint(0, 2**32 - 1)
    run_name = f"{args.env_id}__{args.exp_name}__{args.seed}__{int(time.time())}"
    if args.track:
        import wandb

        wandb.init(
            project=args.wandb_project_name,
            entity=args.wandb_entity,
            sync_tensorboard=True,
            config=vars(args),
            name=run_name,
            monitor_gym=True,
            save_code=True,
        )
    writer = SummaryWriter(f"runs/{run_name}")
    writer.add_text(
        "hyperparameters",
        "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(args).items()])),
    )

    # TRY NOT TO MODIFY: seeding
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.backends.cudnn.deterministic = args.torch_deterministic

    device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")

    # env setup
    torch.manual_seed(args.seed)
    envs = make_env(args.env_id, args.seed, 0, args.capture_video, run_name)()
    reservoir_size = 256
    input_space = spaces.Box(low=-1, high=1, shape=(envs.observation_space.shape[0]+reservoir_size,), dtype=np.float32)

    actor =  ESNLayer(envs, 256).to(device)
    qf1 = QNetwork(envs).to(device)
    qf2 = QNetwork(envs).to(device)
    qf1_target = QNetwork(envs).to(device)
    qf2_target = QNetwork(envs).to(device)
    target_actor = ESNLayer(envs, 256).to(device)
    target_actor.load_state_dict(actor.state_dict())
    qf1_target.load_state_dict(qf1.state_dict())
    qf2_target.load_state_dict(qf2.state_dict())
    q_optimizer = optim.Adam(list(qf1.parameters()) + list(qf2.parameters()), lr=args.learning_rate)
    actor_optimizer = optim.Adam(list(actor.parameters()), lr=args.learning_rate)

    envs.observation_space.dtype = np.float32



    rb = ReplayBuffer(
        args.buffer_size,
        input_space,
        envs.action_space,
        device,
        handle_timeout_termination=False,
    )
    start_time = time.time()

    # TRY NOT TO MODIFY: start the game
    obs, _ = envs.reset(seed=args.seed)
    hidden_state = np.zeros((reservoir_size,))
    input = np.concat((obs,hidden_state))
    all_trajectory = {}
    for global_step in range(args.total_timesteps):

        # SAVE MODEL: save model per 2000 global steps
        model_dir = f"runs/{run_name}/models"
        os.makedirs(model_dir, exist_ok=True)
        if global_step % 2000 == 0 or global_step == args.total_timesteps:
            model_path = f"runs/{run_name}/models/{global_step}steps.cleanrl_model"
            torch.save(actor.state_dict(), model_path)


        obs = input[:envs.observation_space.shape[0]]
        hidden_state = input[envs.observation_space.shape[0]:]

        # INFERENCE:
        if global_step % 8000 == 0 or global_step == args.total_timesteps-1:
            args.initial_positions = [(2,2), (2,10), (2,18), (10,2), (10,18), (18,2), (18,10), (18,18)]
            trajectory = inference(actor, hidden_state, args)
            all_trajectory[f'{global_step}'] = trajectory


        # ALGO LOGIC: put action logic here
        if global_step < args.learning_starts:
            actions = envs.action_space.sample()
        else:
            with torch.no_grad():
                actions = actor(torch.Tensor(obs).to(device), torch.Tensor(hidden_state).to(device))
                actions = actions.cpu().numpy().clip(envs.action_space.low, envs.action_space.high)
                hidden_state, next_hidden_state = actor.get_hiddenstate()
                hidden_state, next_hidden_state = hidden_state.cpu(), next_hidden_state.cpu()
                

        # TRY NOT TO MODIFY: execute the game and log data.
        next_obs, rewards, terminations, truncations, infos = envs.step(actions)
        input = input = np.concat((obs,hidden_state))
        next_input = np.concat((next_obs, next_hidden_state))
        # TRY NOT TO MODIFY: record rewards for plotting purposes
        if "final_info" in infos or terminations:
            writer.add_scalar("charts/episodic_return", infos["episode"]["r"], global_step)
            writer.add_scalar("charts/episodic_length", infos["episode"]["l"], global_step)     

        # TRY NOT TO MODIFY: save data to reply buffer; handle `final_observation`
        next_obs = next_obs.copy()
        if truncations:
            real_next_obs = infos["final_observation"]
        rb.add(input, next_input, actions, rewards, terminations, infos)

        # TRY NOT TO MODIFY: CRUCIAL step easy to overlook
        input = next_input

        # ALGO LOGIC: training.
        if global_step > args.learning_starts:
            data = rb.sample(args.batch_size)
            with torch.no_grad():
                next_observations = data.next_observations[:,:envs.observation_space.shape[0]]
                next_hidden_states = data.next_observations[:,envs.observation_space.shape[0]:]
                observations = data.observations[:,:envs.observation_space.shape[0]]
                hidden_states = data.observations[:,envs.observation_space.shape[0]:]
                next_state_actions = target_actor(next_observations, next_hidden_states).clamp(
                    envs.action_space.low[0], envs.action_space.high[0]
                )
                qf1_next_target = qf1_target(next_observations, next_state_actions)
                qf2_next_target = qf2_target(next_observations, next_state_actions)
                min_qf_next_target = torch.min(qf1_next_target, qf2_next_target)
                next_q_value = data.rewards.flatten() + (1 - data.dones.flatten()) * args.gamma * (min_qf_next_target).view(-1)

            qf1_a_values = qf1(observations, data.actions).view(-1)
            qf2_a_values = qf2(observations, data.actions).view(-1)
            qf1_loss = F.mse_loss(qf1_a_values, next_q_value)
            qf2_loss = F.mse_loss(qf2_a_values, next_q_value)
            qf_loss = qf1_loss + qf2_loss

            # optimize the model
            q_optimizer.zero_grad()
            qf_loss.backward()
            q_optimizer.step()

            if global_step % args.policy_frequency == 0:
                actor_loss = -qf1(observations, actor(observations, hidden_states)).mean()
                actor_optimizer.zero_grad()
                actor_loss.backward()
                actor_optimizer.step()
                actor.hebbtrace_update(hidden_states)
                

                # update the target network
                for param, target_param in zip(actor.parameters(), target_actor.parameters()):
                    target_param.data.copy_(args.tau * param.data + (1 - args.tau) * target_param.data)
                for param, target_param in zip(qf1.parameters(), qf1_target.parameters()):
                    target_param.data.copy_(args.tau * param.data + (1 - args.tau) * target_param.data)
                for param, target_param in zip(qf2.parameters(), qf2_target.parameters()):
                    target_param.data.copy_(args.tau * param.data + (1 - args.tau) * target_param.data)

            if global_step % 100 == 0:
                writer.add_scalar("losses/qf1_values", qf1_a_values.mean().item(), global_step)
                writer.add_scalar("losses/qf2_values", qf2_a_values.mean().item(), global_step)
                writer.add_scalar("losses/qf1_loss", qf1_loss.item(), global_step)
                writer.add_scalar("losses/qf2_loss", qf2_loss.item(), global_step)
                writer.add_scalar("losses/qf_loss", qf_loss.item() / 2.0, global_step)
                writer.add_scalar("losses/actor_loss", actor_loss.item(), global_step)
    envs.close()
    writer.close()

    with open(f"runs/{run_name}/all_trajectory.pkl", "wb") as file:
        pickle.dump(all_trajectory, file)